## 5.A Lab Replication

### 1.1 Cross validation for linear model

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from random import sample
import random
random.seed(14023799)
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset = load_boston()
boston_housing = pd.DataFrame(dataset.data, columns= map(str.lower, dataset.feature_names))
boston_housing['medv'] = dataset.target

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

#### 10-fold Cross Validation

In [4]:
num = list(range(len(boston_housing)))
ran = sample(num, int(len(num)*0.8))
rem = [i for i in num if i not in ran] 

boston_train = boston_housing.iloc[ran, :]
boston_test = boston_housing.iloc[rem, :]

X_train = boston_train.iloc[:,:-1]
y_train = boston_train.iloc[:,-1:]

X_test = boston_test.iloc[:,:-1]
y_test = boston_test.iloc[:,-1:]

df1 = X_train[['indus','rm']]

In [5]:
mse = make_scorer(mean_squared_error)
model_1 = LinearRegression()
cross_val_score(model_1, X_train, y_train, cv = 10,scoring = mse).mean()

24.10330742840943

In [6]:
cross_val_score(model_1, df1, y_train, cv = 10,scoring = mse).mean()

40.653207866034634

#### Leave one out cross validation

In [7]:
cross_val_score(model_1, X_train, y_train, cv = X_train.shape[0],scoring = mse).mean()

24.311952804117674

In [8]:
cross_val_score(model_1, df1, y_train, cv = X_train.shape[0], scoring = mse).mean()

40.84407580572506

#### 10-fold Cross Validation Using MAE

In [9]:
def MAD_cost(pi, r):
    return((abs(pi-r)).mean())


my_scorer = make_scorer(MAD_cost, greater_is_better=True)


cross_val_score(model_1, X_train, y_train, scoring = my_scorer, cv = 10).mean()

3.4512930506123896

In [10]:
cross_val_score(model_1, df1, y_train, scoring = my_scorer, cv = 10).mean()

4.361620759842027

#### Cross Validation for search optimal tuning parameter in LASSO

In [11]:
#use 10-fold cross validation to pick lambda

from sklearn.linear_model import LassoCV

boston_train = boston_housing.iloc[: , :-1]
model_lassocv = LassoCV(cv = 10).fit(boston_train, boston_housing['medv'])

print('The variable coefficients provided by the lasso are:')
coef = pd.Series(model_lassocv.coef_, index = boston_train.columns)
coef

The variable coefficients provided by the lasso are:


crim      -0.074266
zn         0.049454
indus     -0.000000
chas       0.000000
nox       -0.000000
rm         1.804385
age        0.011333
dis       -0.813244
rad        0.272284
tax       -0.015425
ptratio   -0.742872
b          0.008926
lstat     -0.703654
dtype: float64

In [12]:
print("The chosen alpha is: ")
model_lassocv.alpha_

The chosen alpha is: 


0.7248204283772595

In [13]:
test_pred = model_lassocv.predict(boston_train)
MAD_cost(test_pred,boston_housing['medv'])

3.486781991328123

## 5A Exercise

For boston housing dataset, conduct 2-fold CV by cv.glm to calculate the mean square error of full model, and a reduced model with indus and rm only. In terms of the CV criterion, which of the two models do you prefer?

#### MSE of full model

In [14]:
full_model = LinearRegression()
cross_val_score(full_model, X_train, y_train, cv = 2,scoring = mse).mean()

25.704174120940735

#### Cross validation score of full model

In [15]:
cross_val_score(full_model, X_train, y_train, cv = 2).mean()

0.694720332094095

#### MSE of reduced model

In [16]:
reduced_model = LinearRegression()
cross_val_score(reduced_model, df1, y_train, cv = 2,scoring = mse).mean()

41.86556939011739

#### Cross validation score of reduced model

In [17]:
cross_val_score(reduced_model, X_train, y_train, cv = 2).mean()

0.694720332094095

#### The MSE for the full model is less so we can prefer the full model over reduced model